In [ ]:
# PCA pre pumpy (farba podľa čísla pumpy a dátumu)

# axis_to_magnitude(x: pd.DataFrame, axis: list, window_size: int)
# models.cross_cuts_3d(X, y)
# project_classifier_map_plot()

In [ ]:
# PCA pre kompresory (farba podľa čísla kompresoru a dátumu)

In [ ]:
# Label je označenie jednotky - KSB-1, ..., ...

In [ ]:
# Vyber features s najväčšou varianciou (3)

In [ ]:
# CLustrovanie features

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Callable
import tsfel
import numpy as np
from sklearn.feature_selection import mutual_info_classif, f_classif
from scipy.stats import gmean

import sys
sys.path.append('../')
from vibrodiagnostics import datasets, selection, discovery
from sklearn.preprocessing import MinMaxScaler


PARTS = 5
DIR = '../../inspections/datacentres/shc3/'
FILENAMES = [
    'k3_1.tsv', 'k3_2.tsv', 'k3_3.tsv',     # Bad
    'k5_1.tsv', 'k5_2.tsv', 'k5_3.tsv'      # Good
]

FEATURES_PATH = os.path.join(DIR, 'features')
FILENAMES = [os.path.join(DIR, filename) for filename in FILENAMES]

In [ ]:
def boxplot_features(dataset):
    FEATURES_FILENAME = os.path.join(FEATURES_PATH, dataset)
    features = pd.read_csv(FEATURES_FILENAME)
    # Compute magnitude of feature vectors
    names = features['name']

    # Choose window size
    if dataset == selection.FREQ_FEATURES_PATH:
        window_size = 2**12
        features = features.loc[:,features.columns.str.endswith(f'_{window_size}')]
        features.columns = features.columns.str.extract(r'([\w\_]+)_(\w+)$')[0]

    features['ac_unit'] = names.str.extract(r'(k\d+)').astype('category')

    axis = ['x', 'y', 'z']
    feature_names = np.unique(features.drop(columns=['ac_unit']).columns.str.extract(r'[a-z]+_(\w+)').dropna().T.to_numpy())
    result = pd.DataFrame()
    for name in feature_names:              
        vector_dims = [f'{dim}_{name}' for dim in axis]
        result[name] = features[vector_dims].apply(np.linalg.norm, axis=1)

    result_labeled = result.copy()
    result_labeled['ac_unit'] = features['ac_unit']

    # Show boxplots, split by AC unit
    result.boxplot(figsize=(15, 5))
    plt.show()
    result_labeled.boxplot(figsize=(20, 5), layout=(2, 6), by='ac_unit', sharey=False)
    plt.show()

    # MinMax scaled result
    scaler = MinMaxScaler()
    result_scaled = pd.DataFrame()
    result_scaled[result.columns] = scaler.fit_transform(result)

    result_labeled = result_scaled.copy()
    result_labeled['ac_unit'] = features['ac_unit']

    # Show boxplots, split by AC unit
    result_scaled.boxplot(figsize=(15, 5))
    plt.show()
    result_labeled.boxplot(figsize=(20, 5), layout=(2, 6), by='ac_unit', sharey=False)
    plt.show()
    return result_scaled, features['ac_unit']

In [ ]:
X_t, Y_t = boxplot_features(selection.TIME_FEATURES_PATH)

In [ ]:
X_f, Y_f = boxplot_features(selection.FREQ_FEATURES_PATH)

In [ ]:
METRICS_OFFLINE = (selection.corr_classif, f_classif, mutual_info_classif)

def batch_feature_ranking(X: pd.DataFrame, Y: pd.DataFrame) -> pd.DataFrame:
    metric_ranks = pd.DataFrame()  # Independent scores

    for metric_name, metric in zip(('corr', 'f_stat', 'mi'), METRICS_OFFLINE):
        scores = metric(X, Y)
        if isinstance(scores, tuple):
            scores = scores[0]
        leaderboard = (
            pd.DataFrame(zip(X.columns, scores), columns=['feature', 'score'])
            .set_index('feature')
            .sort_values(by='score', ascending=False)
        )
        metric_ranks[metric_name] = leaderboard
    
    ranks = metric_ranks.rank(axis='rows', method='first', ascending=False)
    return metric_ranks, ranks.apply(gmean, axis=1).sort_values().to_frame(name='rank')

In [ ]:
scores, ranks = batch_feature_ranking(X_t, Y_t)
scores

In [ ]:
ranks

In [ ]:
scores, ranks = batch_feature_ranking(X_f, Y_f)
scores

In [ ]:
ranks